In [1]:
import pandas as pd
path = '/rds/general/user/ms2524/home/amexproject/parquet_files/train/train_'
base_cba = pd.read_parquet(path + "base.parquet", columns=['case_id','WEEK_NUM','target'])

In [ ]:
import pandas as pd

def missing_value_percentages(df):
    missing_percent = df.isnull().mean() * 100
    return missing_percent.sort_values(ascending=False)
def detect_high_cardinality(df, threshold=50):
    cat_cols = df.select_dtypes(include=['object', 'category']).columns
    high_card = [col for col in cat_cols if df[col].nunique() > threshold]
    return high_card

# Debitcard, Deposit and other table

In [3]:
debitcard_1 = pd.read_parquet(path +'debitcard_1.parquet')
deposit_1 = pd.read_parquet(path +'deposit_1.parquet')
other_1 = pd.read_parquet(path +'other_1.parquet')

### Not all ids appear in these tables, its a small subset of ids so is it worth using these tables

# Deposit

In [13]:
deposit_1["case_id"].nunique()

105111

In [20]:
deposit_1

,case_id,amount_416A,contractenddate_991D,num_group1,openingdate_313D
0,225,0.00000,NaT,0,2016-08-16
1,331,260.37400,2018-03-18,0,2015-03-19
2,358,0.00000,NaT,0,2014-09-02
3,390,203.60200,2017-09-30,1,2015-10-01
4,390,223.68001,NaT,2,2016-06-08
...,...,...,...,...,...
145081,2703430,0.00000,NaT,7,2016-02-25
145082,2703430,22918.00600,NaT,8,2016-11-26
145083,2703439,219.98401,NaT,0,2016-11-25
145084,2703453,0.00000,NaT,0,2014-08-18


In [19]:
import pandas as pd

def process_deposit_accounts(df):
    df['contractenddate_991D'] = pd.to_datetime(df['contractenddate_991D'], errors='coerce')
    def custom_agg(group):
        total_amount = group['amount_416A'].sum()
        contract_closed_flag = int(group['contractenddate_991D'].notna().all())
        return pd.Series({
            'amount_416A': total_amount,
            'is_closed': contract_closed_flag
        })

    result = df.groupby('case_id').apply(custom_agg).reset_index()
    return result
process_deposit_accounts(deposit_1)

/rds/general/user/ms2524/ephemeral/ipykernel_367002/3904144058.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = df.groupby('case_id').apply(custom_agg).reset_index()


,case_id,amount_416A,is_closed
0,225,0.00000,0.0
1,331,260.37400,1.0
2,358,0.00000,0.0
3,390,212175.81201,0.0
4,445,23735.93800,0.0
...,...,...,...
105106,2703426,1181.73610,1.0
105107,2703427,250.76399,1.0
105108,2703430,22918.00600,0.0
105109,2703439,219.98401,0.0


# Debitcard

In [14]:
debitcard_1["case_id"].nunique()

111772

In [23]:
debitcard_1[debitcard_1["openingdate_857D"].isnull()]

,case_id,last180dayaveragebalance_704A,last180dayturnover_1134A,last30dayturnover_651A,num_group1,openingdate_857D
35,1661,NaN,NaN,NaN,0,None
36,1661,NaN,NaN,NaN,1,None
37,1661,NaN,NaN,NaN,2,None
38,1661,NaN,NaN,NaN,3,None
43,1887,NaN,NaN,NaN,0,None
...,...,...,...,...,...,...
157242,2703197,1.880000,12607.4,0.0,0,None
157246,2703220,0.000000,23800.0,0.0,0,None
157254,2703297,0.000000,71000.0,0.0,0,None
157261,2703329,0.633334,29997.0,0.0,0,None


In [24]:
missing_value_percentages(debitcard_1)

last30dayturnover_651A           92.955589
last180dayturnover_1134A         92.955589
last180dayaveragebalance_704A    92.234047
openingdate_857D                  8.080635
case_id                           0.000000
num_group1                        0.000000
dtype: float64

### We will drop the time feature 'openingdate_857D' and aggregate the remaining feaatures across ids.

In [31]:
debitcard_1.drop(columns='num_group1').groupby('case_id', as_index=False).sum(numeric_only=True)

,case_id,last180dayaveragebalance_704A,last180dayturnover_1134A,last30dayturnover_651A
0,225,0.0,0.0,0.0
1,331,0.0,0.0,0.0
2,358,0.0,0.0,0.0
3,390,0.0,0.0,0.0
4,445,0.0,0.0,0.0
...,...,...,...,...
111767,2703426,0.0,0.0,0.0
111768,2703427,0.0,0.0,0.0
111769,2703430,0.0,0.0,0.0
111770,2703439,0.0,0.0,0.0


## Other

In [21]:
other_1["case_id"].nunique()

51109

In [26]:
other_1.drop(columns='num_group1').groupby('case_id', as_index=False).sum(numeric_only=True)


,case_id,amtdebitincoming_4809443A,amtdebitoutgoing_4809440A,amtdepositbalance_4809441A,amtdepositincoming_4809444A,amtdepositoutgoing_4809442A
0,43801,12466.6010,12291.2000,914.2000,0.0,304.80002
1,43991,3333.4001,3273.4001,0.0000,0.0,0.00000
2,44001,10000.0000,10000.0000,0.0000,0.0,0.00000
3,44053,0.0000,0.0000,2586.4001,0.0,88.80000
4,44130,63.8000,60.8000,0.0000,0.0,0.00000
...,...,...,...,...,...,...
51104,2703443,344.6000,343.4000,0.0000,0.0,0.00000
51105,2703448,0.0000,0.0000,2184.2000,0.0,18.00000
51106,2703450,0.0000,0.0000,0.0000,0.0,0.00000
51107,2703451,27500.0000,27477.6000,0.0000,0.0,0.00000


# Person

In [35]:
person_1 = pd.read_parquet(path +'person_1.parquet')
person_2 = pd.read_parquet(path +'person_2.parquet')

In [36]:
person_1

,case_id,birth_259D,birthdate_87D,childnum_185L,contaddr_district_15M,contaddr_matchlist_1032L,contaddr_smempladdr_334L,contaddr_zipcode_807M,education_927M,empl_employedfrom_271D,...,registaddr_district_1083M,registaddr_zipcode_184M,relationshiptoclient_415T,relationshiptoclient_642T,remitter_829L,role_1084L,role_993L,safeguarantyflag_411L,sex_738L,type_25L
0,0,1986-07-01,None,NaN,P88_18_84,False,False,P167_100_165,P97_36_170,2017-09-15,...,P88_18_84,P167_100_165,None,None,None,CL,None,True,F,PRIMARY_MOBILE
1,0,None,None,NaN,a55475b1,None,None,a55475b1,a55475b1,None,...,a55475b1,a55475b1,SPOUSE,None,False,EM,None,None,None,PHONE
2,0,None,None,NaN,a55475b1,None,None,a55475b1,a55475b1,None,...,a55475b1,a55475b1,COLLEAGUE,SPOUSE,False,PE,None,None,None,PHONE
3,0,None,None,NaN,a55475b1,None,None,a55475b1,a55475b1,None,...,a55475b1,a55475b1,None,COLLEAGUE,None,PE,None,None,None,PHONE
4,1,1957-08-01,None,NaN,P103_93_94,False,False,P176_37_166,P97_36_170,2008-10-29,...,P103_93_94,P176_37_166,None,None,None,CL,None,True,M,PRIMARY_MOBILE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2973986,2703451,None,None,NaN,a55475b1,None,None,a55475b1,a55475b1,None,...,a55475b1,a55475b1,None,None,None,CL,None,None,None,HOME_PHONE
2973987,2703452,1977-08-01,None,NaN,P133_44_167,False,False,P59_150_74,a55475b1,None,...,P19_11_176,P11_15_81,None,None,None,CL,None,False,M,PRIMARY_MOBILE
2973988,2703453,1950-02-01,None,NaN,P123_6_84,False,False,P46_103_143,a55475b1,None,...,P123_6_84,P46_103_143,None,None,None,CL,None,False,F,PRIMARY_MOBILE
2973989,2703453,None,None,NaN,a55475b1,None,None,a55475b1,a55475b1,None,...,a55475b1,a55475b1,None,None,None,CL,None,None,None,HOME_PHONE


In [38]:
missing_value_percentages(person_2)

empls_employedfrom_796D     99.649692
relatedpersons_role_762T    98.252049
addres_role_871L            95.882099
addres_district_368M         0.000000
case_id                      0.000000
conts_role_79M               0.000000
addres_zip_823M              0.000000
empls_economicalst_849M      0.000000
empls_employer_name_740M     0.000000
num_group1                   0.000000
num_group2                   0.000000
dtype: float64

In [45]:
person_1["case_id"].nunique()

1526659

In [ ]:
person_1[person_1["num_group1"]==0]["familystate_447L"].value_counts(dropna=False)

In [46]:
missing_value_percentages(person_1[person_1["num_group1"]==0])

childnum_185L                100.000000
relationshiptoclient_642T    100.000000
remitter_829L                100.000000
relationshiptoclient_415T    100.000000
housingtype_772L             100.000000
maritalst_703L                99.999934
gender_992L                   99.183970
isreference_387L              99.183970
birthdate_87D                 99.183970
role_993L                     99.183970
housetype_905L                93.396168
empl_industry_691L            65.792230
empl_employedtotal_800L       65.394040
empl_employedfrom_271D        62.879661
familystate_447L              52.274018
contaddr_smempladdr_334L       0.028887
contaddr_matchlist_1032L       0.028887
safeguarantyflag_411L          0.000131
contaddr_district_15M          0.000000
case_id                        0.000000
birth_259D                     0.000000
language1_981M                 0.000000
incometype_1044T               0.000000
empladdr_zipcode_114M          0.000000
empladdr_district_926M         0.000000


In [55]:
detect_high_cardinality(person_1[person_1["num_group1"]==0])

['birth_259D',
 'birthdate_87D',
 'contaddr_district_15M',
 'contaddr_zipcode_807M',
 'empl_employedfrom_271D',
 'empladdr_district_926M',
 'empladdr_zipcode_114M',
 'registaddr_district_1083M',
 'registaddr_zipcode_184M']

Feature                      Missing amount

childnum_185L                100.000000
relationshiptoclient_642T    100.000000
remitter_829L                100.000000
relationshiptoclient_415T    100.000000
housingtype_772L             100.000000
maritalst_703L                99.999934
gender_992L                   99.183970
isreference_387L              99.183970
birthdate_87D                 99.183970
role_993L                     99.183970
housetype_905L                93.396168

High cardinality features:

empladdr_zipcode_114M          0.000000
empladdr_district_926M         0.000000
contaddr_zipcode_807M          0.000000
registaddr_zipcode_184M        0.000000
registaddr_district_1083M      0.000000


These features just have one value so theres no value in keeping them:

persontype_792L                0.000000
persontype_1072L               0.000000
personindex_1023L              0.000000



empl_industry_691L            65.792230 - High Cardinality
empl_employedtotal_800L       65.394040 - Useful features
empl_employedfrom_271D        62.879661 - Similar to empl_employedtotal_800L 



Selected features:
empl_employedtotal_800L 
familystate_447L
contaddr_smempladdr_334L       0.028887
contaddr_matchlist_1032L       0.028887
safeguarantyflag_411L          0.000131
birth_259D                     0.000000 convert to age
language1_981M                 0.000000
incometype_1044T               0.000000
education_927M                 0.000000
mainoccupationinc_384A         0.000000
role_1084L                     0.000000
sex_738L                       0.000000
type_25L                       0.000000

In [90]:
missing_value_percentages(person_2[person_2["num_group1"]==0])

empls_employedfrom_796D     99.999932
relatedpersons_role_762T    99.999932
addres_role_871L            98.859917
addres_district_368M         0.000000
case_id                      0.000000
conts_role_79M               0.000000
addres_zip_823M              0.000000
empls_economicalst_849M      0.000000
empls_employer_name_740M     0.000000
num_group1                   0.000000
num_group2                   0.000000
dtype: float64

In [99]:
detect_high_cardinality(person_2[(person_2["num_group1"]==0) & (person_2["num_group2"]==0)])

['addres_district_368M', 'addres_zip_823M']

In [92]:
person_2["case_id"].nunique()

1435105

In [97]:
person_1["case_id"].nunique()

1526659

In [95]:
person_2[person_2["num_group1"]==0]["empls_employer_name_740M"].value_counts(dropna=False)

empls_employer_name_740M
a55475b1      1463927
P207_71_72          1
Name: count, dtype: int64

High cardinality features:
'addres_district_368M', 'addres_zip_823M'

High missing features:
empls_employedfrom_796D     99.999932
relatedpersons_role_762T    99.999932
addres_role_871L            98.859917


Useless features:
empls_employer_name_740M - just 1 value for second category

Features elected:
empls_economicalst_849M
conts_role_79M

# Previous applications

In [103]:
applprev_1_0 = pd.read_parquet(path +'applprev_1_0.parquet')
applprev_1_1 = pd.read_parquet(path +'applprev_1_1.parquet')
applprev_2 = pd.read_parquet(path + 'applprev_2.parquet')

In [105]:
applprev = pd.concat([applprev_1_0, applprev_1_1])

In [33]:
applprev_1_0.columns

Index(['case_id', 'actualdpd_943P', 'annuity_853A', 'approvaldate_319D',
       'byoccupationinc_3656910L', 'cancelreason_3545846M', 'childnum_21L',
       'creationdate_885D', 'credacc_actualbalance_314A',
       'credacc_credlmt_575A', 'credacc_maxhisbal_375A',
       'credacc_minhisbal_90A', 'credacc_status_367L',
       'credacc_transactions_402L', 'credamount_590A', 'credtype_587L',
       'currdebt_94A', 'dateactivated_425D', 'district_544M', 'downpmt_134A',
       'dtlastpmt_581D', 'dtlastpmtallstes_3545839D', 'education_1138M',
       'employedfrom_700D', 'familystate_726L', 'firstnonzeroinstldate_307D',
       'inittransactioncode_279L', 'isbidproduct_390L', 'isdebitcard_527L',
       'mainoccupationinc_437A', 'maxdpdtolerance_577P', 'num_group1',
       'outstandingdebt_522A', 'pmtnum_8L', 'postype_4733339M',
       'profession_152M', 'rejectreason_755M', 'rejectreasonclient_4145042M',
       'revolvingaccount_394A', 'status_219L', 'tenor_203L'],
      dtype='object')

In [143]:
missing_value_percentages(applprev)

revolvingaccount_394A          95.452790
credacc_minhisbal_90A          95.096000
credacc_status_367L            95.096000
credacc_maxhisbal_375A         95.096000
credacc_transactions_402L      95.096000
credacc_actualbalance_314A     95.096000
isdebitcard_527L               92.905080
byoccupationinc_3656910L       76.487083
dtlastpmt_581D                 72.791898
dtlastpmtallstes_3545839D      61.961906
employedfrom_700D              59.553946
childnum_21L                   54.542376
dateactivated_425D             48.142248
maxdpdtolerance_577P           47.436622
approvaldate_319D              46.127853
familystate_726L               36.682496
outstandingdebt_522A           34.604279
currdebt_94A                   34.424138
firstnonzeroinstldate_307D      9.998224
tenor_203L                      8.455743
pmtnum_8L                       8.455743
annuity_853A                    3.842121
credamount_590A                 3.098355
credtype_587L                   3.098355
downpmt_134A    

In [106]:

detect_high_cardinality(applprev)

['approvaldate_319D',
 'cancelreason_3545846M',
 'creationdate_885D',
 'dateactivated_425D',
 'district_544M',
 'dtlastpmt_581D',
 'dtlastpmtallstes_3545839D',
 'employedfrom_700D',
 'firstnonzeroinstldate_307D',
 'profession_152M']

In [138]:
applprev["status_219L"].value_counts(dropna=False)

status_219L
D       2667953
K       2658434
A        715907
T        441632
N         30430
Q          7477
S          2567
L           959
H           472
None         66
P            59
R            23
Name: count, dtype: int64

In [126]:
detect_high_cardinality(applprev)

['approvaldate_319D',
 'cancelreason_3545846M',
 'creationdate_885D',
 'dateactivated_425D',
 'district_544M',
 'dtlastpmt_581D',
 'dtlastpmtallstes_3545839D',
 'employedfrom_700D',
 'firstnonzeroinstldate_307D',
 'profession_152M']

In [119]:
applprev_2[(applprev_2["num_group1"]==0)&(applprev_2["num_group2"]==0)]

,case_id,cacccardblochreas_147M,conts_type_509L,credacc_cards_status_52L,num_group1,num_group2
0,2,None,PRIMARY_MOBILE,None,0,0
4,3,None,PHONE,None,0,0
7,4,None,PRIMARY_MOBILE,None,0,0
9,5,None,PRIMARY_MOBILE,None,0,0
10,6,None,PRIMARY_MOBILE,None,0,0
...,...,...,...,...,...,...
14075405,2703450,a55475b1,PRIMARY_MOBILE,None,0,0
14075434,2703451,a55475b1,PRIMARY_MOBILE,None,0,0
14075451,2703452,a55475b1,PRIMARY_MOBILE,None,0,0
14075457,2703453,a55475b1,PRIMARY_MOBILE,None,0,0


In [140]:
applprev[applprev["num_group1"]==0]

,case_id,actualdpd_943P,annuity_853A,approvaldate_319D,byoccupationinc_3656910L,cancelreason_3545846M,childnum_21L,creationdate_885D,credacc_actualbalance_314A,credacc_credlmt_575A,...,num_group1,outstandingdebt_522A,pmtnum_8L,postype_4733339M,profession_152M,rejectreason_755M,rejectreasonclient_4145042M,revolvingaccount_394A,status_219L,tenor_203L
0,2,0.0,640.2,None,NaN,a55475b1,0.0,2013-04-03,NaN,0.0,...,0,NaN,24.0,a55475b1,a55475b1,a55475b1,a55475b1,NaN,D,24.0
2,3,0.0,6140.0,None,NaN,P94_109_143,NaN,2019-01-07,NaN,0.0,...,0,NaN,12.0,a55475b1,a55475b1,P94_109_143,a55475b1,NaN,D,12.0
3,4,0.0,2556.6,None,NaN,P24_27_36,NaN,2019-01-08,NaN,0.0,...,0,NaN,24.0,a55475b1,a55475b1,a55475b1,a55475b1,NaN,T,24.0
4,5,0.0,NaN,None,NaN,P85_114_140,NaN,2019-01-16,NaN,NaN,...,0,NaN,NaN,a55475b1,a55475b1,a55475b1,a55475b1,NaN,T,NaN
6,6,0.0,1773.8,None,NaN,P94_109_143,NaN,2018-09-12,NaN,0.0,...,0,NaN,11.0,a55475b1,a55475b1,P94_109_143,a55475b1,NaN,D,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2638266,2703450,0.0,2102.2,2019-10-09,NaN,a55475b1,NaN,2019-10-09,NaN,0.0,...,0,0.0,12.0,P177_117_192,a55475b1,a55475b1,a55475b1,NaN,K,12.0
2638275,2703451,0.0,0.0,2019-09-30,NaN,a55475b1,NaN,2019-09-30,NaN,0.0,...,0,0.0,NaN,P46_145_78,a55475b1,a55475b1,a55475b1,780594500.0,A,NaN
2638283,2703452,0.0,9048.0,None,NaN,P180_60_137,NaN,2020-08-21,NaN,0.0,...,0,NaN,14.0,P67_102_161,a55475b1,a55475b1,a55475b1,NaN,T,14.0
2638284,2703453,0.0,0.0,2019-12-18,NaN,a55475b1,NaN,2019-12-18,NaN,0.0,...,0,0.0,NaN,P46_145_78,a55475b1,a55475b1,a55475b1,780825900.0,A,NaN
